In [7]:
from tqdm import tqdm

In [ ]:
# pip install top2vec

# pip install top2vec[sentence_encoders]
# pip install top2vec[sentence_transformers]
# pip install top2vec[indexing]

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from top2vec import Top2Vec
import top2vec

In C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
pd.set_option('max_colwidth', 400)

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [4]:
df_all = pd.read_hdf('outputs/budzety_clean.h5', key='df_all')
df_all = df_all.reset_index()
del df_all['index']

In [5]:
df_all

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,...,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,...,False,False,False,False,False,False,projektistatus,False,False,False
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",3,"[doposażenie, park, pomiędzy, estakada, kwiatkowskiego, wieżowiec, ul, dantyszka, siłownia, zewnętrzny, element, mały, architektura, pergola, bujać, ławka, itp]",False,False,False,False,False,...,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowskiego wieżowiec ul dantyszka siłownia zewnętrzny element mały architektura pergola bujać ławka itp,False,True,False
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0,"[cykliczny, warsztat, senior, dzielnica, obłuże, rozwijać, pamięć, koncentracja]",False,False,False,False,False,...,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże rozwijać pamięć koncentracja,False,False,False
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,...,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0,"[montaż, słup, oświetleniowy, chodnik, ul, niski, odcinek, pomiędzy, ul, bohater, getto, warszawski, hodowlany]",False,False,False,False,False,...,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odcinek pomiędzy ul bohater getto warszawski hodowlany,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,...,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False
23026,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,...,False,False,False,False,False,False,znany mieszkanców,False,False,False
23027,2020,KRAKÓW,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultura w zieleni,1,"[zrobić, skwer, fabryczny, edukacja, kultura, zieleń]",False,False,False,False,False,...,False,False,False,False,False,False,zrobić skwer fabryczny edukacja kultura zieleń,False,False,False
23028,2020,KRAKÓW,Zróbmy sobie Park,1,"[zrobić, park]",False,False,False,False,False,...,False,False,False,False,False,False,zrobić park,False,False,False


In [6]:
docs = list(df_all.loc[:, 'names'].values)
docs = [str(elem) for elem in docs] 

In [8]:
from sentence_transformers import SentenceTransformer, models

In [27]:
from sentence_transformers import SentenceTransformer, models

def get_sentence_embeddings(dataset):
    
    # model do wskazania z https://huggingface.co/models
    # https://github.com/boorism/medium-clustering/blob/main/clustering.ipynb
    word_embedding_model = models.Transformer('allegro/herbert-base-cased', max_seq_length=384)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    sentence_embeddings = []

    for index in range(len(dataset)):
        sentence = dataset.names[index]  #names to nazwa kolumny zawierającej nazwy budżetów
        embedding = model.encode(sentence)
        sentence_embeddings.append(embedding)
    
    return sentence_embeddings 

emb = get_sentence_embeddings(df_all)

model = Top2Vec(docs, embedding_model=emb)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [13]:
# !!!!!! WAŻNE !!!!!!
#
# Jeżeli masz w  folderze models plik:
# 'main_top2vec_topic.model' to nie uruchamiaj
# tej komórki 

model = Top2Vec(docs, embedding_model=emb)
# model.save('models/main_top2vec_topic.model')

2022-01-01 11:21:00,433 - top2vec - INFO - Pre-processing documents for training
2022-01-01 11:21:01,073 - top2vec - INFO - Downloading universal-sentence-encoder-multilingual model
INFO:absl:Using C:\Users\mariu\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual/3: 60.01MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual/3: 130.01MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual/3: 200.01MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-multilingual/3: 260.01MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-multilingual/3, Total size: 266.88MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'.
2022-01-01 11:22:11,

In [9]:
# Ładowanie wcześniej wytrenowanego modelu
# model = Top2Vec.load('models/main_top2vec_topic.model')

In [14]:
# Łącznie mamy tyle tematów
model.get_num_topics()

223

In [15]:
# Ograniczamy liczbę tematów
N_TOPICS = 20
model.hierarchical_topic_reduction(N_TOPICS)

[[172,
  120,
  61,
  95,
  192,
  169,
  154,
  170,
  199,
  118,
  85,
  194,
  160,
  203,
  34,
  208,
  139,
  110,
  58,
  131,
  99,
  215,
  114,
  213,
  111,
  147,
  168,
  214,
  0],
 [84,
  18,
  185,
  60,
  126,
  75,
  27,
  152,
  157,
  93,
  205,
  23,
  136,
  64,
  107,
  54,
  36,
  175,
  55,
  135,
  207,
  103,
  191,
  206,
  178,
  45,
  24,
  119,
  48,
  35,
  108,
  190,
  32],
 [51, 197, 19, 115, 184, 193, 28, 44, 70, 183, 127, 94, 167, 200, 4],
 [71, 79, 3],
 [41, 11],
 [143, 42, 201, 128, 13],
 [12, 138, 73, 106, 92, 146, 159, 9],
 [17,
  33,
  83,
  105,
  176,
  202,
  153,
  69,
  211,
  218,
  166,
  57,
  25,
  177,
  125,
  65,
  156,
  142,
  198,
  104,
  87,
  113,
  210,
  86,
  121,
  217,
  122,
  182,
  76,
  109,
  171,
  21,
  164,
  56,
  50,
  117,
  149,
  38],
 [134,
  22,
  133,
  40,
  148,
  43,
  150,
  137,
  173,
  101,
  144,
  102,
  15,
  97,
  130,
  163,
  222,
  209,
  49],
 [5],
 [216, 16, 10],
 [1],
 [219, 140, 100, 116

In [16]:
# Wyciągamy numery tematów, słowa należące do tematów oraz wyniki podobieństwa słów
topic_words, word_scores, topic_nums = model.get_topics(reduced=True)

In [17]:
# Podgląd słów stanowiących temat 10
topic_words[10]

array(['dziecko', 'dzieciecy', 'młodziez', 'rodzice', 'młody', 'rodzinny',
       'rodzina', 'dorosły', 'edukacyjny', 'praca', 'letni', 'mały',
       'szkolny', 'przedszkole', 'zabawa', 'przedszkolny',
       'ogolnodostepny', 'edukacja', 'szkoła', 'ogrod', 'gimnastyczny',
       'rekreacyjny', 'skatepark', 'plac', 'ławeczka', 'przedszkolak',
       'siatkowka', 'sportowy', 'kreatywny', 'rowerzysta', 'wypoczynkowy',
       'osobowy', 'zewnetrzny', 'nowoczesny', 'nowosc', 'rekreacja',
       'miec', 'dojscie', 'sportowo', 'wygodny', 'zajecie', 'fitness',
       'ochotniczy', 'zajecia', 'szkolenie', 'nowosci', 'kultura',
       'kulturalny', 'osoba', 'stadion'], dtype='<U14')

In [18]:
# Najbardziej podobne słowa do wskazanego
words, word_scores = model.similar_words(keywords=["wodny"], keywords_neg=[], num_words=20)

for word, score in zip(words, word_scores):
    print(f"{word} {score}")

woda 0.838851283180854
zatoka 0.6945461935839659
powietrze 0.6778668380534503
teren 0.6630604908190048
łodzki 0.6555663859286431
skwer 0.6476976318283256
oswietlenia 0.6445162866561306
łodz 0.6432325209172749
swiat 0.6400009651197567
istniec 0.634119092169782
sprzet 0.6337221429195345
wesoły 0.6332554849049016
szczecina 0.6327775931234204
wybieg 0.6186416857956325
solankowy 0.612653667256593
siłownia 0.6126497523478243
staw 0.6101594648961153
oswietlenie 0.6047516812804103
terenowy 0.6045406142190566
czesc 0.6045245906661477


In [19]:
# Wyszukaj dokumenty najbardziej związane z danym słowem
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["siłownia"], num_docs=20)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 15752, Score: 0.7622712989395707
-----------
zgiełk
-----------

Document: 15875, Score: 0.7622712989395707
-----------
zgiełk
-----------

Document: 5255, Score: 0.7549945291856592
-----------
teren
-----------

Document: 16197, Score: 0.7495777421847128
-----------
podwórkowy
-----------

Document: 16250, Score: 0.7268754660303092
-----------
bieńczycach
-----------

Document: 13291, Score: 0.7045399768954979
-----------
tężnia
-----------

Document: 11981, Score: 0.7045399768954979
-----------
tężnia
-----------

Document: 2902, Score: 0.7045399768954979
-----------
tężnia
-----------

Document: 5257, Score: 0.6937085053573491
-----------
podwórko
-----------

Document: 11069, Score: 0.6867481044962986
-----------
zalewa
-----------

Document: 16180, Score: 0.6842607958698126
-----------
bagier
-----------

Document: 18701, Score: 0.6721886125129459
-----------
brać
-----------

Document: 5887, Score: 0.671684319079414
-----------
pturek
-----------

Document: 805, Score: 

In [20]:
df_all['nazwa'].iloc[21652]

'Siłownia w Prusie'

In [21]:
df_all['nazwa'].iloc[11889]

'Sala gimnastyczna w SP 3.'

In [22]:
# Pokaż 4 dokumenty (o największym prawdopodobieństwie) należące do tematu nr 18
SELECTED_TOPIC = 18
documents, document_scores, document_nums = model.search_documents_by_topic(topic_num=SELECTED_TOPIC, num_docs=4, reduced=True)
    
result_df = df_all.loc[document_nums]
result_df["document_scores"] = document_scores
print(f'Temat: {SELECTED_TOPIC}')
print(*topic_words[18])
result_df[['nazwa', 'names', 'document_scores']]

Temat: 18
biblioteka czytelnik audiobook toaleta wydawniczy siatkowka edukacyjny uczen miec zakup nowosci nowoczesny akademia centrum koszykowka ul kompleks teatr osiedle warsztat wypoczynkowy bezpłatny osiedlowy uliczny ogrod dojscie mieszkancow kino multimedialny nowosc sala miejski komputerowy wieko warsztaty dzieciecy szkoła edukacja wypozyczalnia budynek doposazenie gorka społeczny kosciuszki zakatek szatnia wolny szkolny wokoł rekreacyjny


,nazwa,names,document_scores
15126,ZAKUP KSIĄŻEK I AUDIOBOOKÓW DLA UŻYTKOWNIKÓW BIBLIOTEKI NA KOZINACH - FILII 21 BIBLIOTEKI MIEJSKIEJ.,zakup książka audiobook użytkownik biblioteka kozina filia biblioteka miejski,0.879708
12267,Zakup książek i audiobooków dla użytkowników biblioteki na Kozinach - Filii 21 Biblioteki Miejskiej.,zakup książka audiobook użytkownik biblioteka kozina filia biblioteka miejski,0.879708
13991,- Zakup książek - nowości wydawniczych dla czytelników Biblioteki Miejskiej w Łodzi Filii Nr 39.,zakup książka nowość wydawniczy czytelnik biblioteka miejski łódź filia,0.877513
13995,- Zakup nowości książkowych dla użytkowników Filii Bibliotecznej nr 47 Biblioteki Miejskiej w Łodzi.,zakup nowość książkowy użytkownik filia biblioteczny biblioteka miejski łódź,0.875586


In [23]:
# Indeksy wszystkich dokumentów
indeksy = df_all.index.tolist()

In [24]:
# Buduj tematy dla dokumentów
documents_topics = model.get_documents_topics(indeksy, reduced=True)

In [144]:
# documents_topics składa się z 4 list:
# 0 - Numer tematu dla poszczególnego dokumentu
# 1 - Semantyczne podobieństwo dokumentu do tematu (odl. cosinusowa wektora dokumentu
#  i tematu)
# 2 - Top 50 słów danego tematu
# 3 - Odległość cosinusowa tematów z każdym z top 50 słów

4

In [25]:
# Zapisanie tematów do DataFrame'a
df_all['topics'] = documents_topics[0]

In [27]:
df_all['topics_desc'] = pd.DataFrame(documents_topics[2]).agg(' '.join, axis=1).agg(lambda x: x.split())

In [28]:
df_all['topics'].value_counts()

0     2560
1     1754
2     1472
3     1420
4     1205
5     1203
6     1119
7     1089
8     1041
9     1013
10    1001
11     997
12     994
13     962
14     945
15     933
16     919
17     827
18     799
19     777
Name: topics, dtype: int64

In [220]:
df_all.to_hdf('outputs/budzety_clean_topics_top2vec.h5', key='df_all')

In [16]:
df.shape

(23030, 63)

In [18]:
import pandas as pd

In [20]:
df = pd.read_hdf('outputs/budzety_clean_topics_top2vec.h5')

In [35]:
for n in range(0, 20):
    print(topic_words[n])

['ogolnodostepny' 'solankowy' 'zewnetrzny' 'pomiedzy' 'sprzet'
 'doswietlenie' 'piekny' 'trawnik' 'siatkowka' 'wspolny' 'paweł'
 'wypozyczalnia' 'ławeczka' 'skwer' 'dojscie' 'ruchowy' 'wokoł' 'wesoły'
 'zajecia' 'zakatek' 'skrzyzowanie' 'siłownia' 'uporzadkowanie' 'kielec'
 'przedszkolak' 'sciezka' 'zajecie' 'ksiazka' 'łaka' 'szatnia' 'podworko'
 'łaczyc' 'wozek' 'slaski' 'łodz' 'oswietlenie' 'szczecina' 'srodmiescie'
 'nawierzchnia' 'wzdłuz' 'staw' 'swietlny' 'kieszonkowy' 'wieko' 'sciezek'
 'chmurka' 'nowosci' 'przedszkole' 'istniec' 'stawa']
['dzielnicowy' 'dzielnica' 'osiedlowy' 'wypoczynkowy' 'osiedle'
 'wypozyczalnia' 'rekreacyjny' 'wypoczynek' 'miejski' 'mieszkancow'
 'ogolnodostepny' 'koncert' 'teatr' 'lokalny' 'rekreacja' 'zabytkowy'
 'odpoczynek' 'stadion' 'teatralny' 'ulica' 'otoczenie' 'nawierzchnia'
 'miasteczko' 'szatnia' 'kulturalny' 'kultura' 'ławeczka' 'centrum'
 'obywatelski' 'miasto' 'mieszkaniec' 'plac' 'park' 'siatkowka' 'cmentarz'
 'plenerowy' 'ogrod' 'miejsce' 'f